In [1]:
import numpy as np
import pandas as pd
import joblib
from os import listdir
from os.path import isfile, join
from commonFunctions import musicFeatureExtraction
from commonFunctions import displayVAgraph

#spremenljivke
from commonFunctions import subfolderName
from commonFunctions import showResults
subfolder = subfolderName

onlyfiles = [f for f in listdir(subfolder) if isfile(join(subfolder, f)) and ".mp3" in f]

allFeatures = []
for fileName in onlyfiles:
    print(fileName)
    data = musicFeatureExtraction(subfolder+str(fileName))
    allFeatures.append([fileName, data])

featuresdf = pd.DataFrame(allFeatures, columns=['id','features'])
featuresdf = featuresdf.set_index(['id'])


005.mp3
019.mp3
084.mp3
179.mp3
192.mp3
299.mp3


In [ ]:

indexesFeatures = 5
regressionType = "RF"
selectionType = "corelation"
VA = ['valence', 'arousal']

for selectVA in VA:
    
    selectedFeatIds = pd.read_pickle('Dataset/indexes/'+selectionType+'_'+str(indexesFeatures)+'_'+selectVA+'_indexes.pkl')
    indexes = np.array(selectedFeatIds['indexes'].tolist()) # read indexes file
    
    X_pred = np.array(featuresdf['features'].tolist()) # input
    X_pred_filtered = X_pred[:, indexes]# filter features by index
    X_pred = X_pred_filtered
    
    model = joblib.load('./models/FULL/modelRF_'+selectVA+'_5_corelation.joblib')
    predictions = model.predict(X_pred)
    
    if selectVA == 'valence':
        predictions_valence = predictions
        npValence = np.round(np.array(predictions),2)
    else:
        predictions_arousal = predictions
        npArousal = np.round(np.array(predictions),2)


In [ ]:


# od tukaj do printa je samo za prikaz !
combined = np.vstack((npValence, npArousal)).T

names = [x.replace('.mp3', '') for x in onlyfiles]

dataValenceArousal = pd.DataFrame(data=combined, index=names, columns=['valence', 'arousal']) 
print(dataValenceArousal.sort_index())

# displayVAgraph(valence, arousal, names, min, max)
displayVAgraph( predictions_valence, predictions_arousal, names, -1, 1 )

if showResults:
    from commonFunctions import rezultatiTestData
    rezultatiTestData(predictions_valence, predictions_arousal)